In [1]:
import pandas as pd
import os

df = pd.DataFrame()
"""
for json_file in os.listdir("dataset/jsonl/"):
    df = pd.concat([df, pd.read_json("dataset/jsonl/" + json_file, lines=True)])
    print(f"Added {json_file} to dataframe.")
"""
df = pd.read_json('dataset/jsonl/en-US.jsonl', lines=True)
df["locale"] = df["locale"].apply(lambda x: x.split("-")[0])

In [2]:
%run -i "preprocess_lang.ipynb"
%run -i "preprocess_nolang.ipynb"
%run -i "evaluation.ipynb"

In [3]:
from toolz.functoolz import pipe

params = [
    remove_punctuation,
    lowercase,
    tokenize
]

df = pipe(
    df,
    *params
)


In [4]:
import numpy as np

def tokens_to_mean_vector(embeddings, tokens):
    # convert tokens to embedding vectors, up to sequence_len tokens
    vec = []
    n = 0
    i = 0
    while i < len(tokens):   # while there are tokens
        try:
            vec.append(embeddings.get_vector(tokens[i]))
            n += 1
        except KeyError:
            True   # simply ignore out-of-vocabulary tokens
        finally:
            i += 1
    
    # return the mean of vec
    return np.mean(vec, axis=0)    


In [5]:
from gensim.models import Word2Vec

model = Word2Vec.load(f'D:\models\en.model')
print(f"Loaded en model.")

MemoryError: Unable to allocate 2.24 GiB for an array with shape (600000000,) and data type float32

In [ ]:
for lang in df['locale'].unique():
    print(f"Adding {lang} embeddings to dataframe.")
    # apply tokens_to_mean_vector to each utterance with locale == lang
    for i, row in df[df['locale'] == lang].iterrows():
        df.at[i, 'utt'] = tokens_to_mean_vector(model.wv, row['utt'])
    print(f"Added {lang} embeddings to dataframe. Deleting model.")


Adding en embeddings to dataframe.
Added en embeddings to dataframe. Deleting model.


In [7]:

NUM_DIM = 50
input_cols = [f'utt_{i}' for i in range(NUM_DIM)]
for i in range(300):
    df[f'utt_{i}'] = df['utt'].apply(lambda x: x[i])

C:\Users\nrtc\AppData\Local\Temp\ipykernel_22136\751542648.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'utt_{i}'] = df['utt'].apply(lambda x: x[i])
C:\Users\nrtc\AppData\Local\Temp\ipykernel_22136\751542648.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'utt_{i}'] = df['utt'].apply(lambda x: x[i])
C:\Users\nrtc\AppData\Local\Temp\ipykernel_22136\751542648.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Co

In [8]:
df.head()

,id,locale,partition,scenario,intent,utt,annot_utt,worker_id,utt_text,utt_0,...,utt_290,utt_291,utt_292,utt_293,utt_294,utt_295,utt_296,utt_297,utt_298,utt_299
0,0,en,test,alarm,alarm_set,"[0.02374037, -0.10971783, 0.1179467, 0.0553602...",wake me up at [time : five am] [date : this week],1,"(w, a, k, e, , m, e, , u, p, , a, t, , f, ...",0.023740,...,0.129545,-0.015426,-0.287220,0.072414,0.021808,0.018839,-0.011701,0.204422,-0.033505,-0.001701
1,1,en,train,alarm,alarm_set,"[0.02635483, -0.063925914, 0.12291399, 0.06185...",wake me up at [time : nine am] on [date : friday],1,"(w, a, k, e, , m, e, , u, p, , a, t, , n, ...",0.026355,...,0.097372,-0.013342,-0.299295,0.061895,0.008075,0.021580,0.008930,0.172068,-0.047982,-0.022944
2,2,en,train,alarm,alarm_set,"[0.0149873, -0.03628377, 0.022763085, 0.090717...",set an alarm for [time : two hours from now],1,"(s, e, t, , a, n, , a, l, a, r, m, , f, o, ...",0.014987,...,0.079397,0.070716,-0.050164,-0.034825,-0.045124,-0.023538,-0.031646,0.199483,-0.006318,-0.014939
3,3,en,test,audio,audio_volume_mute,"[-0.059513863, -0.079797, -0.011928498, 0.0888...",quiet,1,"(q, u, i, e, t)",-0.059514,...,0.080874,-0.001509,-0.056021,-0.000028,0.019974,-0.035629,0.007633,0.078737,-0.063585,0.063456
4,4,en,train,audio,audio_volume_mute,"[-0.017824091, -0.035481222, 0.052026093, 0.09...",olly quiet,1,"(o, l, l, y, , q, u, i, e, t)",-0.017824,...,0.020328,0.056281,-0.137947,-0.009079,0.014584,-0.010109,0.043367,0.111444,-0.053675,-0.028138


In [9]:
output_cols = ['intent']
averages = [None, "macro", "weighted", "micro", "samples"]

In [10]:
training_inputs = df[df['partition'] != 'test'][input_cols].values
testing_inputs = df[df['partition'] == 'test'][input_cols].values
training_classes = df[df['partition'] != 'test'][output_cols].values
testing_classes = df[df['partition'] == 'test'][output_cols].values

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler #fixed import

scaler = MinMaxScaler()
training_inputs = scaler.fit_transform(training_inputs)
testing_inputs = scaler.transform(testing_inputs)

nb = MultinomialNB(alpha=0.1)

nb.fit(training_inputs, training_classes)

C:\Users\nrtc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB(alpha=0.1)

In [13]:
evaluate_model(nb, testing_inputs, testing_classes, output_cols)

Evaluation metrics for MultinomialNB
MultinomialNB's default score metric: 0.07128446536650974
Classification report
                          precision    recall  f1-score   support

             alarm_query     1.0000    0.0000    0.0000        34
            alarm_remove     1.0000    0.0000    0.0000        21
               alarm_set     1.0000    0.0000    0.0000        41
       audio_volume_down     1.0000    0.0000    0.0000        11
       audio_volume_mute     1.0000    0.0000    0.0000        32
      audio_volume_other     1.0000    0.0000    0.0000         6
         audio_volume_up     1.0000    0.0000    0.0000        13
          calendar_query     1.0000    0.0000    0.0000       126
         calendar_remove     1.0000    0.0000    0.0000        67
            calendar_set     0.0703    1.0000    0.1314       209
          cooking_recipe     1.0000    0.0000    0.0000        72
        datetime_convert     1.0000    0.0000    0.0000        15
          datetime_query

In [14]:
from xgboost import XGBClassifier

xgboost = XGBClassifier(subsample=0.7, n_estimators=200, min_child_weight=3, max_depth=3, max_delta_step=1, learning_rate=0.05, gamma=0.5, booster='gblinear')

xgboost.fit(training_inputs, training_classes)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59], got ['alarm_query' 'alarm_remove' 'alarm_set' 'audio_volume_down'
 'audio_volume_mute' 'audio_volume_other' 'audio_volume_up'
 'calendar_query' 'calendar_remove' 'calendar_set' 'cooking_query'
 'cooking_recipe' 'datetime_convert' 'datetime_query' 'email_addcontact'
 'email_query' 'email_querycontact' 'email_sendemail' 'general_greet'
 'general_joke' 'general_quirky' 'iot_cleaning' 'iot_coffee'
 'iot_hue_lightchange' 'iot_hue_lightdim' 'iot_hue_lightoff'
 'iot_hue_lighton' 'iot_hue_lightup' 'iot_wemo_off' 'iot_wemo_on'
 'lists_createoradd' 'lists_query' 'lists_remove' 'music_dislikeness'
 'music_likeness' 'music_query' 'music_settings' 'news_query'
 'play_audiobook' 'play_game' 'play_music' 'play_podcasts' 'play_radio'
 'qa_currenc

In [ ]:
evaluate_model(xgboost, testing_inputs, testing_classes, output_cols)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None)

knn.fit(training_inputs, training_classes)

In [ ]:
evaluate_model(knn, testing_inputs, testing_classes, output_cols)